In [164]:
import pandas as pd
from datasets import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from private_transformers import PrivacyEngine

In [165]:
from datasets import get_dataset_split_names, load_dataset
get_dataset_split_names("dair-ai/emotion")

Using the latest cached version of the module from C:\Users\Saad\.cache\huggingface\modules\datasets_modules\datasets\dair-ai--emotion\cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd (last modified on Tue Jan  2 15:11:31 2024) since it couldn't be found locally at dair-ai/emotion, or remotely on the Hugging Face Hub.


['train', 'validation', 'test']

In [166]:
from transformers import AutoTokenizer

In [301]:
dataset = load_dataset("dair-ai/emotion", split="train")

Using the latest cached version of the module from C:\Users\Saad\.cache\huggingface\modules\datasets_modules\datasets\dair-ai--emotion\cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd (last modified on Tue Jan  2 15:11:31 2024) since it couldn't be found locally at dair-ai/emotion, or remotely on the Hugging Face Hub.


In [302]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

In [303]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

In [304]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'sep_token': '[SEP]'})
tokenizer.add_special_tokens({'bos_token': '[BOS]'})

1

In [305]:
tokenized = tokenizer(
    ["I enjoy walking with my cute dog", "This is a sentence"],
    padding=True
)
for key, value in tokenized.items():
    print(key, value)

input_ids [[40, 2883, 6155, 351, 616, 13779, 3290], [1212, 318, 257, 6827, 50256, 50256, 50256]]
attention_mask [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0]]


In [306]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [240]:
test_row = f'{tokenizer.bos_token}positive{tokenizer.sep_token}{dataset[0]["text"]}'
print(test_row)

[BOS]positive[SEP]i didnt feel humiliated


In [241]:
tokenizer(test_row)

{'input_ids': [50258, 24561, 50257, 72, 42547, 1254, 42659], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [242]:
tokenizer(dataset[0]["text"])

{'input_ids': [72, 42547, 1254, 42659], 'attention_mask': [1, 1, 1, 1]}

In [307]:
LABELS = [
    "sadness",
    "joy",
    "love",
    "anger",
    "fear",
    "suprise"
]

def tokenization(example):
    # print(f"I have {len(example['text'])} rows in example")
    # print(example)

    for idx, row in enumerate(example["text"]):
        example["text"][idx] = f'{tokenizer.bos_token}{LABELS[example["label"][idx]]}{tokenizer.sep_token}{row}{tokenizer.eos_token}'
     
    out_dict = tokenizer(example["text"])

    out_dict["text"] = example["text"]

    return out_dict


dataset = dataset.map(tokenization, batched=True,remove_columns=["label"])

In [308]:
dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
dataset.format['type']

'torch'

In [309]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = dataset.with_format("torch", device=device)
dataset[1]

{'text': '[BOS]sadness[SEP]i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake<|endoftext|>',
 'input_ids': tensor([50258,    82,   324,  1108, 50257,    72,   460,   467,   422,  4203,
           523, 23292,   284,   523, 28911, 17836,   655,   422,   852,  1088,
          2130,   508, 16609,   290,   318, 21693, 50256], device='cuda:0'),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1], device='cuda:0')}

In [311]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

In [312]:
data_collator = DataCollatorForLanguageModeling(
  tokenizer=tokenizer, mlm=False)

In [313]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [314]:
trainer.train()

  0%|          | 0/20000 [00:00<?, ?it/s]

{'loss': 59.5716, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 58.2042, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 54.909, 'learning_rate': 3e-06, 'epoch': 0.01}
{'loss': 49.1693, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 38.503, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 24.077, 'learning_rate': 6e-06, 'epoch': 0.03}
{'loss': 11.0836, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.04}
{'loss': 8.1561, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.04}
{'loss': 7.3841, 'learning_rate': 9e-06, 'epoch': 0.04}
{'loss': 6.2416, 'learning_rate': 1e-05, 'epoch': 0.05}
{'loss': 5.8744, 'learning_rate': 1.1000000000000001e-05, 'epoch': 0.06}
{'loss': 5.4415, 'learning_rate': 1.2e-05, 'epoch': 0.06}
{'loss': 4.8463, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.07}
{'loss': 4.6269, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.07}
{'loss': 4.5079, 'learning_rate': 1.5e-05, 'epoch': 0.07}
{'loss': 4.335, 'le

Checkpoint destination directory ./results\checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 3.6936, 'learning_rate': 5e-05, 'epoch': 0.25}
{'loss': 3.7472, 'learning_rate': 4.997435897435897e-05, 'epoch': 0.26}
{'loss': 3.8301, 'learning_rate': 4.994871794871795e-05, 'epoch': 0.26}
{'loss': 3.7392, 'learning_rate': 4.992307692307693e-05, 'epoch': 0.27}
{'loss': 3.6216, 'learning_rate': 4.9897435897435904e-05, 'epoch': 0.27}
{'loss': 3.7108, 'learning_rate': 4.9871794871794874e-05, 'epoch': 0.28}
{'loss': 3.6001, 'learning_rate': 4.984615384615385e-05, 'epoch': 0.28}
{'loss': 3.7779, 'learning_rate': 4.982051282051282e-05, 'epoch': 0.28}
{'loss': 3.5406, 'learning_rate': 4.979487179487179e-05, 'epoch': 0.29}
{'loss': 3.7191, 'learning_rate': 4.9769230769230775e-05, 'epoch': 0.29}
{'loss': 3.6334, 'learning_rate': 4.9743589743589746e-05, 'epoch': 0.3}
{'loss': 3.6737, 'learning_rate': 4.971794871794872e-05, 'epoch': 0.3}
{'loss': 3.754, 'learning_rate': 4.969230769230769e-05, 'epoch': 0.31}
{'loss': 3.954, 'learning_rate': 4.966666666666667e-05, 'epoch': 0.32}
{'loss':

Checkpoint destination directory ./results\checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


...

Checkpoint destination directory ./results\checkpoint-5500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 3.2223, 'learning_rate': 3.717948717948718e-05, 'epoch': 2.75}
{'loss': 3.3054, 'learning_rate': 3.7153846153846156e-05, 'epoch': 2.75}
{'loss': 3.2389, 'learning_rate': 3.7128205128205126e-05, 'epoch': 2.76}
{'loss': 3.1716, 'learning_rate': 3.7102564102564104e-05, 'epoch': 2.77}
{'loss': 3.189, 'learning_rate': 3.707692307692308e-05, 'epoch': 2.77}
{'loss': 3.2667, 'learning_rate': 3.705128205128206e-05, 'epoch': 2.77}
{'loss': 3.3121, 'learning_rate': 3.702564102564103e-05, 'epoch': 2.78}
{'loss': 3.2388, 'learning_rate': 3.7e-05, 'epoch': 2.79}
{'loss': 3.1329, 'learning_rate': 3.6974358974358975e-05, 'epoch': 2.79}
{'loss': 3.2746, 'learning_rate': 3.6948717948717945e-05, 'epoch': 2.79}
{'loss': 3.2748, 'learning_rate': 3.692307692307693e-05, 'epoch': 2.8}
{'loss': 3.2998, 'learning_rate': 3.68974358974359e-05, 'epoch': 2.81}
{'loss': 3.299, 'learning_rate': 3.6871794871794877e-05, 'epoch': 2.81}
{'loss': 3.2426, 'learning_rate': 3.684615384615385e-05, 'epoch': 2.81}
{'lo

Checkpoint destination directory ./results\checkpoint-6000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 3.411, 'learning_rate': 3.58974358974359e-05, 'epoch': 3.0}
{'loss': 3.1369, 'learning_rate': 3.587179487179487e-05, 'epoch': 3.0}
{'loss': 3.1377, 'learning_rate': 3.584615384615384e-05, 'epoch': 3.01}
{'loss': 3.02, 'learning_rate': 3.582051282051283e-05, 'epoch': 3.02}
{'loss': 3.1164, 'learning_rate': 3.57948717948718e-05, 'epoch': 3.02}
{'loss': 3.0472, 'learning_rate': 3.5769230769230774e-05, 'epoch': 3.02}
{'loss': 3.1012, 'learning_rate': 3.5743589743589744e-05, 'epoch': 3.03}
{'loss': 3.1025, 'learning_rate': 3.571794871794872e-05, 'epoch': 3.04}
{'loss': 2.9535, 'learning_rate': 3.569230769230769e-05, 'epoch': 3.04}
{'loss': 3.1962, 'learning_rate': 3.566666666666667e-05, 'epoch': 3.04}
{'loss': 3.1127, 'learning_rate': 3.5641025641025646e-05, 'epoch': 3.05}
{'loss': 3.0743, 'learning_rate': 3.5615384615384616e-05, 'epoch': 3.06}
{'loss': 3.0239, 'learning_rate': 3.558974358974359e-05, 'epoch': 3.06}
{'loss': 3.0363, 'learning_rate': 3.556410256410256e-05, 'epoch': 3

TrainOutput(global_step=20000, training_loss=3.167433702659607, metrics={'train_runtime': 1791.5575, 'train_samples_per_second': 89.308, 'train_steps_per_second': 11.163, 'train_loss': 3.167433702659607, 'epoch': 10.0})

In [72]:
trainer.save_model("./finetunedmodel")

In [73]:
tokenizer.save_pretrained('./finetunedtokenizer')

('./finetunedtokenizer\\tokenizer_config.json',
 './finetunedtokenizer\\special_tokens_map.json',
 './finetunedtokenizer\\vocab.json',
 './finetunedtokenizer\\merges.txt',
 './finetunedtokenizer\\added_tokens.json')

In [74]:
model = GPT2LMHeadModel.from_pretrained('./finetunedmodel/')
tokenizer = GPT2Tokenizer.from_pretrained('./finetunedtokenizer/')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [315]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50259, bias=False)
)

In [254]:
output = model.generate(
    input_ids, 
    max_length=15,
    num_return_sequences=1,
    temperature=0.9,  # Adjust for creativity
    top_k=50
)
generated_text = tokenizer.decode(output[0])
print(generated_text)

c:\Users\Saad\anaconda3\envs\tenv\Lib\site-packages\transformers\generation\configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[BOS]joy[SEP]i feel like i am being generous with my gifts and i


In [272]:
import random

In [351]:

input_ids = torch.tensor(tokenizer.encode("[BOS]sadness[SEP]")).unsqueeze(0).to(device)  # Empty string as a prompt

output = model.generate(
    input_ids, 
    max_length=50,
    num_return_sequences=1,  # Number of sequences to return
    num_beams=10,             # Number of beams for beam search
    temperature=0.2,
    top_k=25,
    no_repeat_ngram_size=3,
    early_stopping=True
    

    
)

tokenizer.decode(output[0], skip_special_tokens=False)

c:\Users\Saad\anaconda3\envs\tenv\Lib\site-packages\transformers\generation\configuration_utils.py:404: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `25` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'[BOS]sadness[SEP]i feel like i missed out on a lot of valuable information in this blog post but here it is a list of things that you can do to make your life a better place if you want to make it better for yourself and'